In [1]:
party_id = 0

num_parties = 2  ## number of participating parties
dataset = 'mnist'

In [2]:
def get_party_config(party_id):
    party_config = {
        'aggregator':
            {
                'ip': '127.0.0.1',
                'port': 5000
            },
        'connection': {
            'info': {
                'ip': '127.0.0.1',
                'port': 8085 + party_id,
                'id': 'party' + str(party_id),
                'tls_config': {
                    'enable': False
                }
            },
            'name': 'FlaskConnection',
            'path': 'ibmfl.connection.flask_connection',
            'sync': False
        },
        'data': {
            'info': {
                'npz_file': 'examples/data/mnist/random/data_party'+ str(party_id) +'.npz'
            },
            'name': 'MnistKerasDataHandler',
            'path': 'ibmfl.util.data_handlers.mnist_keras_data_handler'
        },
        'local_training': {
            'name': 'LocalTrainingHandler',
            'path': 'ibmfl.party.training.local_training_handler'
        },
        'model': {
            'name': 'KerasFLModel',
            'path': 'ibmfl.model.keras_fl_model',
            'spec': {
                'model_definition': 'examples/configs/keras_classifier/compiled_keras.h5',
                'model_name': 'keras-cnn'
            }
        },
        'protocol_handler': {
            'name': 'PartyProtocolHandler',
            'path': 'ibmfl.party.party_protocol_handler'
        }
    }
    return party_config

In [3]:
from ibmfl.party.party import Party
import tensorflow as tf

party_config = get_party_config(party_id)
party = Party(config_dict=party_config)
party.start()
party.register_party()
party.proto_handler.is_private = False  ## allows sharing of metrics with aggregator

/home/opc/miniconda3/envs/ibm_fl/lib/python3.6/site-packages/google/auth/crypt/_cryptography_rsa.py:22: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography (40.0) will be the last to support Python 3.6.
  import cryptography.exceptions


2023-02-06 20:39:18,182 | 1.1.0 | INFO | ibmfl.util.config                             | Getting Aggregator details from arguments.
2023-02-06 20:39:20,016 | 1.1.0 | INFO | ibmfl.util.config                             | No metrics recorder config provided for this setup.


Using TensorFlow backend.


2023-02-06 20:39:21,907 | 1.1.0 | INFO | ibmfl.util.config                             | No metrics config provided for this setup.
2023-02-06 20:39:21,910 | 1.1.0 | INFO | ibmfl.util.config                             | No evidencia recordeer config provided for this setup.
2023-02-06 20:39:21,953 | 1.1.0 | INFO | ibmfl.util.data_handlers.mnist_keras_data_handler | Loaded training data from examples/data/mnist/random/data_party0.npz
2023-02-06 20:39:22,500 | 1.1.0 | WARNING | tensorflow                                    | From /home/opc/miniconda3/envs/ibm_fl/lib/python3.6/site-packages/ibmfl/model/keras_fl_model.py:59: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

2023-02-06 20:39:22,571 | 1.1.0 | WARNING | tensorflow                                    | From /home/opc/miniconda3/envs/ibm_fl/lib/python3.6/site-packages/ibmfl/model/keras_fl_model.py:60: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2023

2023-02-06 20:39:52,952 | 1.1.0 | INFO | ibmfl.party.training.local_training_handler   | Local training done, generating model update...
5000/5000 [==============================] - 15s 3ms/step
2023-02-06 20:40:23,057 | 1.1.0 | INFO | ibmfl.model.keras_fl_model                    | {'f1 micro': 0.21, 'precision micro': 0.92, 'recall micro': 0.12, 'f1 macro': 0.14, 'precision macro': 0.29, 'recall macro': 0.12, 'f1 weighted': 0.14, 'precision weighted': 0.29, 'recall weighted': 0.12}
2023-02-06 20:40:23,125 | 1.1.0 | INFO | ibmfl.model.keras_fl_model                    | {'loss': 1.70458041267395, 'acc': 0.467, 'f1 micro': 0.21, 'precision micro': 0.92, 'recall micro': 0.12, 'f1 macro': 0.14, 'precision macro': 0.29, 'recall macro': 0.12, 'f1 weighted': 0.14, 'precision weighted': 0.29, 'recall weighted': 0.12}
2023-02-06 20:40:23,131 | 1.1.0 | INFO | ibmfl.party.training.local_training_handler   | {'loss': 1.70458041267395, 'acc': 0.467, 'f1 micro': 0.21, 'precision micro': 0.92, 'rec

...

2023-02-06 20:47:59,808 | 1.1.0 | INFO | werkzeug                                      | 127.0.0.1 - - [06/Feb/2023 20:47:59] "POST /7 HTTP/1.1" 200 -
2023-02-06 20:47:59,875 | 1.1.0 | INFO | ibmfl.party.training.local_training_handler   | Local model updated.
2023-02-06 20:47:59,955 | 1.1.0 | INFO | ibmfl.party.training.local_training_handler   | Local training started...
2023-02-06 20:47:59,962 | 1.1.0 | INFO | ibmfl.model.keras_fl_model                    | Training hps for this round => batch_size: 128, epochs 3, steps_per_epoch 100
Epoch 1/3
200/200 [==============================] - 1s 7ms/step - loss: 0.0406 - acc: 0.9950
Epoch 2/3
200/200 [==============================] - 1s 7ms/step - loss: 0.0273 - acc: 0.9950
Epoch 3/3
200/200 [==============================] - 1s 7ms/step - loss: 0.0247 - acc: 0.9950
2023-02-06 20:48:04,266 | 1.1.0 | INFO | ibmfl.party.training.local_training_handler   | Local training done, generating model update...
5000/5000 [===========================

2023-02-06 20:49:21,397 | 1.1.0 | INFO | ibmfl.party.party_protocol_handler            | finished async request
2023-02-06 20:49:21,397 | 1.1.0 | INFO | ibmfl.party.party_protocol_handler            | Handling async request in a separate thread
2023-02-06 20:49:21,402 | 1.1.0 | INFO | ibmfl.party.party_protocol_handler            | Received request from aggregator
2023-02-06 20:49:21,403 | 1.1.0 | INFO | ibmfl.party.party_protocol_handler            | Received request in with message_type:  7
2023-02-06 20:49:21,456 | 1.1.0 | INFO | ibmfl.party.party_protocol_handler            | Received request in PH 7
2023-02-06 20:49:21,400 | 1.1.0 | INFO | werkzeug                                      | 127.0.0.1 - - [06/Feb/2023 20:49:21] "POST /7 HTTP/1.1" 200 -
2023-02-06 20:49:21,566 | 1.1.0 | INFO | ibmfl.party.training.local_training_handler   | Local model updated.
2023-02-06 20:49:21,574 | 1.1.0 | INFO | ibmfl.party.training.local_training_handler   | Local training started...
2023-02-06 2

2023-02-06 20:50:38,677 | 1.1.0 | INFO | ibmfl.party.party_protocol_handler            | successfully finished async request
2023-02-06 20:50:43,178 | 1.1.0 | INFO | ibmfl.connection.flask_connection             | Request received for path :7
2023-02-06 20:50:45,107 | 1.1.0 | INFO | ibmfl.party.party_protocol_handler            | received a async request
2023-02-06 20:50:45,110 | 1.1.0 | INFO | ibmfl.party.party_protocol_handler            | finished async request
2023-02-06 20:50:45,110 | 1.1.0 | INFO | ibmfl.party.party_protocol_handler            | Handling async request in a separate thread
2023-02-06 20:50:45,115 | 1.1.0 | INFO | ibmfl.party.party_protocol_handler            | Received request from aggregator
2023-02-06 20:50:45,113 | 1.1.0 | INFO | werkzeug                                      | 127.0.0.1 - - [06/Feb/2023 20:50:45] "POST /7 HTTP/1.1" 200 -
2023-02-06 20:50:45,156 | 1.1.0 | INFO | ibmfl.party.party_protocol_handler            | Received request in with message_typ

2023-02-06 20:51:59,462 | 1.1.0 | INFO | ibmfl.model.keras_fl_model                    | {'loss': 0.56914847047925, 'acc': 0.8738, 'f1 micro': 0.88, 'precision micro': 0.88, 'recall micro': 0.87, 'f1 macro': 0.87, 'precision macro': 0.88, 'recall macro': 0.87, 'f1 weighted': 0.88, 'precision weighted': 0.88, 'recall weighted': 0.87}
2023-02-06 20:51:59,465 | 1.1.0 | INFO | ibmfl.party.training.local_training_handler   | {'loss': 0.56914847047925, 'acc': 0.8738, 'f1 micro': 0.88, 'precision micro': 0.88, 'recall micro': 0.87, 'f1 macro': 0.87, 'precision macro': 0.88, 'recall macro': 0.87, 'f1 weighted': 0.88, 'precision weighted': 0.88, 'recall weighted': 0.87}
2023-02-06 20:51:59,469 | 1.1.0 | INFO | ibmfl.party.party_protocol_handler            | successfully finished async request
2023-02-06 20:52:01,577 | 1.1.0 | INFO | ibmfl.connection.flask_connection             | Request received for path :7
2023-02-06 20:52:03,716 | 1.1.0 | INFO | ibmfl.party.party_protocol_handler            

200/200 [==============================] - 1s 6ms/step - loss: 0.0245 - acc: 0.9950
Epoch 3/3
200/200 [==============================] - 1s 7ms/step - loss: 0.0284 - acc: 0.9900
2023-02-06 20:52:59,272 | 1.1.0 | INFO | ibmfl.party.training.local_training_handler   | Local training done, generating model update...
5000/5000 [==============================] - 9s 2ms/step
2023-02-06 20:53:17,670 | 1.1.0 | INFO | ibmfl.model.keras_fl_model                    | {'f1 micro': 0.88, 'precision micro': 0.88, 'recall micro': 0.87, 'f1 macro': 0.87, 'precision macro': 0.88, 'recall macro': 0.87, 'f1 weighted': 0.88, 'precision weighted': 0.89, 'recall weighted': 0.87}
2023-02-06 20:53:17,671 | 1.1.0 | INFO | ibmfl.model.keras_fl_model                    | {'loss': 0.5035947044074536, 'acc': 0.8734, 'f1 micro': 0.88, 'precision micro': 0.88, 'recall micro': 0.87, 'f1 macro': 0.87, 'precision macro': 0.88, 'recall macro': 0.87, 'f1 weighted': 0.88, 'precision weighted': 0.89, 'recall weighted': 0.8

In [ ]:
import keras
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
_, (X_test, Y_test) = mnist.load_data()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import keras
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
sample_count = 100
num_parties = 1
for i in range(10):
   test_filter = np.where(Y_test == i)
   X_test1, Y_test1 = X_test[test_filter], Y_test[test_filter]
   #labels, counts = np.unique(Y_test1, return_counts=True)
   np.random.seed(123)
   rand_v = np.random.randint(0, X_test1.shape[0],sample_count)
   test_digits = X_test1[rand_v]
   test_labels = Y_test1[rand_v]
   labels, counts = np.unique(test_labels, return_counts=True)
   #print('Original lable', labels)
   acc_list = np.array([])
   y_true = np.full(sample_count, i)

       
   y_pred = np.array([])
   for i_samples in range(sample_count):
      pred = party.fl_model.predict(test_digits[i_samples].reshape(1, 28, 28, 1))
      y_pred = np.append(y_pred, pred.argmax())
   acc = accuracy_score(y_true, y_pred) * 100
   #print('y prediction',y_pred)
   #print('y true',y_true)
   #print('accuracy',acc)
   acc_list = np.append(acc_list,acc)

   ind = np.arange(num_parties) 
   fig2 = plt.figure(constrained_layout=True,figsize=(10,10))
   fig2.tight_layout()
   spec2 = gridspec.GridSpec(ncols=6, nrows=4, figure=fig2) 
   f2_ax1 = fig2.add_subplot(spec2[0:1, 0:2])
   f2_ax1.imshow(test_digits[0], cmap='gray')
   plt.axis('off')
   f2_ax2 = fig2.add_subplot(spec2[0:1, 3:])
   labels = [(i+1) for i in range(num_parties)] 
   
   x = np.arange(num_parties)
  

   rects1 = f2_ax2.bar(x, acc_list, width=0.3)
   # Add some text for labels, title and custom x-axis tick labels, etc.
   f2_ax2.set_xlabel('Party ' + str(party_id))
   f2_ax2.set_ylabel('Average Prediction Accuracy \n over '+str(sample_count)+' samples')
   f2_ax2.set_title('Average Prediction Accuracy by parties for label '+str(i))
   f2_ax2.set_xticks(x)
   f2_ax2.set_xticklabels(labels)

In [ ]:
party.stop()